In [2]:
import pandas as pd
import re
import numpy as np
import json

In [9]:
reid_df = pd.read_csv('./log_SGD_continue_1e-4/results.csv')
with open('../RT-DETR/submissions/0.878 autoaug best conf 0.5 no reid lb 0.2973/0.878 autoaug best conf 0.5 no reid lb 0.2973 bbox mapping.json', 'r') as f:
    mapping = json.load(f)
batched_mapping = {}
for k, v in mapping.items():
    k = k.replace('\\', '/')
    batched_mapping['/'.join(k.split('/')[:4] + ['test_old'] + k.split('/')[5:])] = v
mapping = batched_mapping

In [11]:
pattern = re.compile(r'([-\d]+)_c(\d+)')
def combine(row):
    fp = row['file_path'].replace('\\', '/')
    row['Image_ID'] = 'image_' + pattern.search(fp).groups()[-1].zfill(4)
    row['class'] = row['result']
    row['confidence'] = mapping[fp]['confidence']
    row['ymin'] = mapping[fp]['ymin']
    row['xmin'] = mapping[fp]['xmin']
    row['ymax'] = mapping[fp]['ymax']
    row['xmax'] = mapping[fp]['xmax']
    return row

cols = ['Image_ID', 'class', 'confidence', 'ymin', 'xmin', 'ymax', 'xmax']

reid_df_new = reid_df.apply(combine, axis=1)[cols]
reid_df_new.to_csv('submissions/0.878 autoaug best conf 0.5 no reid lb 0.2973_reid_0.7950250502368829_thresh_swinB_RR.csv',index=False)

In [10]:
mapping

{'../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_1.png': {'confidence': '0.962336540222168',
  'ymin': 0.6428440093994141,
  'xmin': 0.13749072551727295,
  'ymax': 0.7769677056206598,
  'xmax': 0.21896100044250488},
 '../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_2.png': {'confidence': '0.9509434700012207',
  'ymin': 0.6679973602294922,
  'xmin': 0.4564194679260254,
  'ymax': 0.8024039374457466,
  'xmax': 0.5443758964538574},
 '../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c0s1_1_3.png': {'confidence': '0.9498826861381531',
  'ymin': 0.6323461744520399,
  'xmin': 0.24650070667266846,
  'ymax': 0.7813136630588108,
  'xmax': 0.3322341203689575},
 '../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c1s1_1_1.png': {'confidence': '0.9609028697013855',
  'ymin': 0.7297207302517361,
  'xmin': 0.15084987878799438,
  'ymax': 0.8831437004937066,
  'xmax': 0.2645244598388672},
 '../RT-DETR/dataset/reid/test_old/bounding_box_test/-1_c1s1_1_2.png': {'confiden